In [3]:
pip install caer canaro


In [4]:
import os
import numpy as np
import cv2 as cv
import caer
import canaro
import gc

In [5]:
IMG_SIZE = (80,80)
channels = 1
char_path = r"../input/the-simpsons-characters-dataset/simpsons_dataset"

In [6]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
char_dict = caer.sort_dict(char_dict, descending = True)

In [7]:
char_dict

In [8]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if(count >= 10):
        break


In [9]:
characters


In [10]:
#training data using caer
train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE = IMG_SIZE, isShuffle = True)

In [11]:
len(train)

In [12]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0],cmap = 'gray')
plt.show()

In [13]:
featureSet, labels = caer.sep_train(train, IMG_SIZE = IMG_SIZE)

In [15]:
from tensorflow.keras.utils import to_categorical
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels,len(characters))

In [16]:
x_train , x_val, y_train, y_val = caer.train_val_split(featureSet,labels,val_ratio = 0.2)

In [19]:
del train
del featureSet
del labels
gc.collect()

In [20]:
BATCH_SIZE = 32
EPOCHS = 10
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)

In [22]:
model = canaro.models.createSimpsonsModel(IMG_SIZE = IMG_SIZE, channels = channels, output_dim = len(characters), loss  ='binary_crossentropy',decay = 1e-6, learning_rate = 0.01, momentum = 0.9, nesterov = True)

In [23]:
model.summary()

In [24]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [25]:
training = model.fit(train_gen, steps_per_epoch = len(x_train)//BATCH_SIZE ,
                    epochs = EPOCHS, validation_data = (x_val, y_val),
                    validation_steps = len(y_val)//BATCH_SIZE, callbacks = callbacks_list)

In [26]:
characters

In [67]:
test_path = '../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/milhouse_van_houten_27.jpg'
img = cv.imread(test_path)
plt.imshow(img)
plt.show()

In [68]:
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [69]:
predictions = model.predict(prepare(img))
print( characters[np.argmax(predictions[0])])